In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler

# Mount Drive, read file
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/aML/airbnb.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# One-hot Encoding for city
df_encoded = pd.get_dummies(df, columns=['city'], prefix='city')

# Binary Encoding for time_period
df_encoded['is_weekend'] = (df_encoded['time_period'] == 'weekends').astype(int)

# Create combined stratification column
df_encoded['strat_column'] = df['city'] + '_' + df['time_period']

#Define features and target
X = df_encoded.drop(['price', 'log_price', 'time_period', 'strat_column'], axis=1)
y = df_encoded['log_price']

#Stratification split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_encoded['strat_column']
)

Linear Regression without Scaling

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
score_test = lr.score(X_test, y_test)
score_train = lr.score(X_train, y_train)
mse_linear = mean_squared_error(y_test, y_pred)

print(f"Linear Regressor Mean Squared Error: {mse_linear:.4f}")
print(f"Linear Regressor Train R^2 Score: {score_train:.4f}")
print(f"Linear Regressor Test R^2 Score: {score_test:.4f}")

Linear Regressor Mean Squared Error: 0.1131
Linear Regressor Train R^2 Score: 0.6680
Linear Regressor Test R^2 Score: 0.6818


Polynomial degrees

In [ ]:
numeric_cols = [
    'person_capacity',
    'cleanliness_rating',
    'guest_satisfaction_overall',
    'city_dist',
    'metro_dist'
]

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

# ColumnTransformer to handle numeric scaling while leaving other columns alone
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('poly_features', PolynomialFeatures()),
    ('lin_reg', LinearRegression())
])

param_grid = {
    'preprocessing__num__scaler': [StandardScaler(), 'passthrough'],
    'poly_features__degree': [1, 2 ]
}


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=1,
    verbose=3,
    error_score='raise'
)

try:

    grid_search.fit(X_train, y_train)

    best_LR_1 = grid_search.best_estimator_
    y_pred = best_LR_1.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    score_train = best_LR_1.score(X_train, y_train)
    score_test = best_LR_1.score(X_test, y_test)

    print("Best parameters found:", grid_search.best_params_)
    print("Best CV R2 score:", grid_search.best_score_)
    print("Train R2 score:", score_train)
    print("Test R2 score:", score_test)
    print("MSE:", mse)

except Exception as e:
    print(f"Error occurred: {str(e)}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END poly_features__degree=1, preprocessing__num__scaler=StandardScaler();, score=0.657 total time=   1.1s
[CV 2/5] END poly_features__degree=1, preprocessing__num__scaler=StandardScaler();, score=0.664 total time=   0.5s
[CV 3/5] END poly_features__degree=1, preprocessing__num__scaler=StandardScaler();, score=0.673 total time=   0.4s
[CV 4/5] END poly_features__degree=1, preprocessing__num__scaler=StandardScaler();, score=0.670 total time=   0.4s
[CV 5/5] END poly_features__degree=1, preprocessing__num__scaler=StandardScaler();, score=0.672 total time=   0.3s
[CV 1/5] END poly_features__degree=1, preprocessing__num__scaler=passthrough;, score=0.657 total time=   0.3s
[CV 2/5] END poly_features__degree=1, preprocessing__num__scaler=passthrough;, score=0.664 total time=   0.3s
[CV 3/5] END poly_features__degree=1, preprocessing__num__scaler=passthrough;, score=0.673 total time=   0.5s
[CV 4/5] END poly_features__degree=